In [1]:
# -*- coding:utf-8 -*-
import codecs # python编码模块
import re # python正则表达式模块
import os
import csv

In [2]:
import jieba.analyse #分词是必须的，分词分析
import matplotlib.pyplot as plt #想画个词云，matplotlib是必须的
import requests #爬虫请求包
from scipy.misc import imread #scipy数据分析包
from wordcloud import WordCloud # 词云

ImportError: cannot import name 'imread' from 'scipy.misc' (D:\App\anaconda3\lib\site-packages\scipy\misc\__init__.py)

In [ ]:
# 带爬取页面的头信息
headers = {
    "Host": "m.weibo.cn",
    "Referer": "https://m.weibo.cn/u/1768409523?uid=1768409523&t=0&luicode=10000011&lfid=100103type%3D1%26q%3D%E5%8D%97%E4%BA%AC%E5%A4%A7%E5%AD%A6",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36",
}

In [4]:
#交待爬虫函数fetch_data需要的参数
url = "https://m.weibo.cn/api/container/getIndex"
# uid和containerid在运行主函数中赋值
params = {"uid": "{uid}",
          "luicode": "10000011",
          #"featurecode": "20000320",
          "type": "uid",
          "value": "1768409523",
          "containerid": "{containerid}",
          "page": "{page}"}

In [5]:
path = os.getcwd()+"/weibo.csv"
#path = "/weibo.csv"
# 注意这里的编码是utf-8-sig，如果是utf-8写入的文件是乱码
csvfile = open(path,'a',encoding='utf-8-sig',newline='')
#clearData(csvfile)
#每一次重新运行程序的时候把上一次保存的数据清空 #是重新运行程序#省区手动清空的多余环节
csvfile.seek(0,0)
csvfile.truncate()
writer = csv.writer(csvfile)
#writer.writerow(('username','source','comment'))
writer.writerow(('text','attitudes_count'))

22

In [6]:
# 这个函数是删除文本中的杂乱信息，只保留文本 ##采用正则表达式
## 按图索骥，查看数据结构，就知道文本的样子
def clean_html(raw_html):
    pattern = re.compile(r'<.*?>|转发微博|//:|Repost|，|？|。|、|分享图片|回复@.*?:|//@.*')
    text = re.sub(pattern, '', raw_html)
    return text

In [7]:
def fetch_data(uid=None, container_id=None):
    """
    抓取数据，并保存到txt文件中
    :return:
    """
    page = 0
    total = 100 ## 这个参数很重要，不然大V的微博会耗时很久试一试20条
    blogs = []
    # 循环页
    for i in range(0, total // 10): ## //是整除的意思 # 就是爬取该号十分之一（页）的内容
        params['uid'] = uid
        params['page'] = str(page)
        params['containerid'] = container_id
        # requests包是爬虫的核心，它的三个参数，已经定义好
        res = requests.get(url, params=params, headers=headers)
        cards = res.json().get("data").get("cards")
        
        #循环每页中的每条微博
        for card in cards:
            # 每条微博的正文内容
            # 打开“开发者工具”看一看，Preview
            if card.get("card_type") == 9:
                text = card.get("mblog").get("text")
                text = clean_html(text)
                blogs.append(text) #计算爬取微博的条数
                attitudes_count = card.get("mblog").get("attitudes_count")
                
                ## 把以上获得的文本数据和点赞数量保存到csv文件中
                ## 注意这里的逻辑，是怎么循环处理的
                writer.writerow((text, attitudes_count))
        page += 1
        print("抓取第{page}页，目前总共抓取了 {count} 条微博".format(page=page, count=len(blogs)))
        # weibo.txt是事先创建的
        #with codecs.open('weibo1.txt', 'w', encoding='utf-8') as f:
        #    f.write("\n".join(blogs))
       

In [8]:
# 主函数（执行）
if __name__ == '__main__':
    fetch_data("768409523", "1076031768409523") ## 给爬虫函数输入两个重要参数


抓取第1页，目前总共抓取了 10 条微博
抓取第2页，目前总共抓取了 20 条微博
抓取第3页，目前总共抓取了 29 条微博
抓取第4页，目前总共抓取了 39 条微博
抓取第5页，目前总共抓取了 49 条微博
抓取第6页，目前总共抓取了 58 条微博
抓取第7页，目前总共抓取了 68 条微博
抓取第8页，目前总共抓取了 75 条微博
抓取第9页，目前总共抓取了 84 条微博
抓取第10页，目前总共抓取了 94 条微博


In [9]:
csvfile.close()